## 字典树提取命名实体

### 定义工具

In [4]:
cd("../data")
include("../src/prefixtree.jl")

remove_subcase

### 初始化字典树

In [5]:
include("../src/readfiles.jl")

# 测试数据
txts = rstrip(read(open("NER_data/valid_ques.txt", "r"), String))
que_and_sub = Dict{String,String}(split(txt, '\t') for txt in split(txts,'\n'))
for (que, sub) in que_and_sub
    que_and_sub[que] = replace(sub, '_'=>' ')
end

In [9]:
# 字典数据
en_words = unique!(vcat(en_subs, collect(keys(ILLs))))
en_words = [replace(word, '_'=>' ') for word in en_words]
zh_words = unique!(vcat(zh_subs, collect(values(ILLs))))
zh_words = [replace(word, '_'=>' ') for word in zh_words]
words = unique!(vcat(en_words, zh_words))
words = [replace(word, '_'=>' ') for word in words]

# 生成字典树
dict, dict_en, dict_zh = PrefixTree(), PrefixTree(), PrefixTree()
for word in words
    add_node!(dict, word)
end
for word in en_words
    add_node!(dict_en, word)
end
for word in zh_words
    add_node!(dict_zh, word)
end

### 抽取命名实体

In [10]:
# NER 的主要函数
# 抽取实体仅出现在头实体上
# 很可能是单向的，下一步 check 利于简化关系抽取
function get_subject(que)
    # 从所有头实体以及 ILLs 中匹配
    subs = remove_subcase(search_valid_word(dict, que))
    length(subs) == 1 && return subs
    # 剩下的大部分是中文问题，带英文关键字
    # 从英文实体以及 ILLs 英文实体中匹配
    subs = remove_subcase(search_valid_word(dict_en, que))
    length(subs) == 1 && return subs
    # 从中文实体以及 ILLs 中文实体中匹配
    subs = remove_subcase(search_valid_word(dict_zh, que))
    length(subs) == 1 && return subs
    return remove_subcase(search_valid_word(dict, que))
end
# 实体带有空格，用字典树进行处理，复杂度 O(实体长度*问题长度)

get_subject (generic function with 1 method)

In [33]:
# 在训练集上测试，valids 为存在唯一解的情况，fails 为多解情况
valids, fails = Tuple{String, String}[], String[]
for (que, _) in que_and_sub
    subs = get_subject(que)
    length(subs) == 1 ? push!(valids, (que, first(subs))) : push!(fails, que)
end
println(length(valids))
count(que_and_sub[que] == sub for (que, sub) in valids)

13187


13185

In [7]:
# 检验唯一解情况的提取结果是否正确 
# 只有两个错误
for (que, sub) in valids
    que_and_sub[que] == sub || println(que)
end

Quel est le fabricant de Choses liées aux Microsoft Lumia 950 XL, un smartphone mobile Windows 10 développé par Microsoft?
黑莓有限公司推出的基于android的slider智能手机BlackBerry Priv的相关产品（作品）属于什么类型


In [18]:
que1 = "Quel est le fabricant de Choses liées aux Microsoft Lumia 950 XL, un smartphone mobile Windows 10 développé par Microsoft?"
println(que_and_sub[que1])
remove_subcase(search_valid_word(dict, que1))

Microsoft Lumia 950 XL


2-element Vector{String}:
 "Windows 10"
 "Microsoft Lumia 950 XL"

In [20]:
que2 = "黑莓有限公司推出的基于android的slider智能手机BlackBerry Priv的相关产品（作品）属于什么类型"
remove_subcase(search_valid_word(dict_zh, que2))
# triple_zh: BlackBerry_Priv	related	BlackBerry_Classic

2-element Vector{String}:
 "智能手机"
 "BlackBerry Priv"

In [12]:
get_subject("加拿大安大略省的一个政党安大略進步保守黨持有席位的议会所属的党派是什么")

2-element Vector{String}:
 "加拿大"
 "安大略進步保守黨"

In [35]:
fails

299-element Vector{String}:
 "日本皇室附属分支的成员華頂博信的父亲的出生地是哪里"
 "非洲南部的一个王国斯威士兰的最大城市的信息来源于谁"
 "受英国天体物理学家约瑟琳·贝尔·伯奈尔影响的人在哪里上班"
 "古埃及女王美麗奈茨的子女的前一任是谁"
 "加拿大安大略省的一个政党安大略進步保守黨持有席位的议会所属的党派是什么"
 "加拿大第23任总理；皮埃尔·特鲁多的长子賈斯汀·杜魯多的父母的父母是谁"
 "英国多塞特的一个城镇和平民教区比明斯特的郡东部是什么地方"
 "美国职业篮球队休斯敦火箭的教练是谁的上一任"
 "英国牛津郡白马谷的一个集镇和平民教区旺塔奇的郡西部是什么地方"
 "荷兰的一个省弗莱福兰省的最大城市的电话号码是哪个国家的"
 "古埃及帝国和迦南叛军之间的一场战斗米吉多战役的将领的爸爸是谁"
 "美国传教士羅啻的国籍所属的区域西北部是什么地方"
 "罗马皇后加拉·普拉西提阿的后裔是哪个帝国的朝代"
 ⋮
 "澳大利亚维多利亚州的一座城市謝珀頓的郡东南边的地方叫什么"
 "乌兹别克斯坦足球运动员法浩特·塔德吉耶夫的国家队的位置属于哪个区划"
 "英格兰林肯郡的一个集镇和平民教区斯利福德的郡西南边的地方叫什么"
 "西班牙的国家公园卡夫雷拉島國家公園的所属群岛的邮政编码类型是什么"
 "谁是捷克足球运动员伊万·哈谢克的出生国家的皇帝"
 "日本王子（1902-1953）秩父宮雍仁親王的父亲的信仰宗教是什么"
 "阿尔巴尼亚足球运动员鲁迪·瓦塔的国家队的举办地被谁租用"
 "新罗女王善德女王的父亲的配偶是哪位"
 "法国国王（1544-1560）弗朗索瓦二世 (法兰西)的母亲的后裔的上一代是什么"
 "越南统治者阮福淍的父亲的下一任是谁"
 "罗马帝国的皇后，君士坦丁大帝的母亲（250-330）聖海倫納的后裔属于哪个帝国"
 "丹麦独立视频游戏开发商和出版商Playdead所在国家有什么代表"

In [26]:
# fails 放在 multi_sols.txt
# 可行策略是取多个匹配中字符长度最大的一个

# 训练集准确率
13183/14077

0.9364921503161185

注记：
   - 加上多匹配 299 个问题，比率还能再提高
   - 加上错位的问题，比率还有进一步提升

In [23]:
# 测试集结果
txts = read(open("valid_data.txt", "r"), String)
ques = String[rstrip(last(split(txt, '\t'))) for txt in split(rstrip(txts), '\n')]

1500-element Vector{String}:
 "where is the constituency of the one who is alongside Felix Chung, a hong kong politician, from?"
 "who is the successor of the parent of Francis Russell, Marquess of Tavistock, an irish politician?"
 "who is the leader of the administrative region to which 热那亚总督 belongs?"
 "who preceded the parent of 富兰克林·德拉诺·罗斯福三世, an american economist?"
 "which draft team does the author of The Way It's Goin' Down, a 1998 single by shaquille o'neal, dj quik, lord tariq and peter gunz, belong to?"
 "who is before the parent of 贾瓦哈拉尔·尼赫鲁, an indian lawyer, statesman, and writer, first prime minister of india (1889-1964)?"
 "where does the location province of Khuut coal mine, a mine in mongolia, belong to?"
 "who does the origin of A's nameryabhata (crater) influence?"
 "what is the genre of the author of They All Went to Mexico, a song performed by carlos santana?"
 "which title does the parent of 富兰克林·德拉诺·罗斯福三世, an american economist, belong to?"
 "what is the parent 

In [24]:
valids, fails = Tuple{String, String}[], String[]
for que in ques
    subs = get_subject(que)
    length(subs) == 1 ? push!(valids, (que, first(subs))) : push!(fails, que)
end

In [25]:
# 存在唯一匹配的数目
println(length(valids))
length(valids)/1500

1463


0.9753333333333334